In [5]:
import findspark
findspark.init()
findspark.find()

'C:\\spark\\spark-2.4.4-bin-hadoop2.7'

In [56]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import*
from pyspark.sql.window import Window
from pyspark.sql.functions import*

In [7]:
conf = SparkConf().setAppName('teste_eleflow').setMaster('local').set('spark.driver.host', 'localhost')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=teste_eleflow, master=local) created by __init__ at <ipython-input-4-39205cd95c47>:2 

In [8]:
spark

In [47]:
# Definindo estrutura para melhorar performance
schema = StructType([
    StructField('mes', IntegerType()),
    StructField('regiao', StringType()),
    StructField('estado', StringType()),
    StructField('municipio', StringType()),
    StructField('produto', StringType()),
    StructField('n_postos_pesquisados', IntegerType()),
    StructField('unidade_medida', StringType()),
    StructField('rev_preco_medio', FloatType()),
    StructField('rev_preco_minimo', FloatType()),
    StructField('rev_preco_maximo', FloatType()),
    StructField('dist_preco_medio', FloatType()),
    StructField('dist_preco_minimo', FloatType()),
    StructField('dist_preco_maximo', FloatType()),
])

sqlContext = SQLContext(spark.sparkContext)
df = spark.read.csv(path='df_limpo.csv', header=True, schema=schema).cache()

In [10]:
df.printSchema()

root
 |-- mes: integer (nullable = true)
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- produto: string (nullable = true)
 |-- n_postos_pesquisados: integer (nullable = true)
 |-- unidade_medida: string (nullable = true)
 |-- rev_preco_medio: float (nullable = true)
 |-- rev_preco_minimo: float (nullable = true)
 |-- rev_preco_maximo: float (nullable = true)
 |-- dist_preco_medio: float (nullable = true)
 |-- dist_preco_minimo: float (nullable = true)
 |-- dist_preco_maximo: float (nullable = true)



In [11]:
# Irá ser calculado tanto valor de distribuição, assim como valor de revenda para todas as questões
# Por isso estou usando exec(). Para economizar linha no código
dict = {'rev' : 'rev_preco_medio', 
        'dist' :'dist_preco_medio'}

for i in dict:
    exec("q1_"+i+"=df.filter(col('"+dict[i]+"') > 0).sort(col('mes'))")
    exec("q1_"+i+"=q1_"+i+".groupBy('mes', 'municipio', 'produto')")
    exec("q1_"+i+"=q1_"+i+".agg(round(avg('"+dict[i]+"'),2))")
    exec("q1_"+i+"=q1_"+i+".withColumnRenamed('round(avg("+i+"_preco_medio), 2)','"+i+"_preco_medio')")

# DataFrame agrupada por mês e médias de valores de cada combustível por cidade
q1_rev.show()

+---+--------------------+----------------+---------------+
|mes|           municipio|         produto|rev_preco_medio|
+---+--------------------+----------------+---------------+
|  1|          ABAETETUBA|ETANOL HIDRATADO|           4.65|
|  1|          ACAILANDIA|ETANOL HIDRATADO|           3.43|
|  1|          ADAMANTINA|ETANOL HIDRATADO|            2.6|
|  1|AGUAS LINDAS DE G...|ETANOL HIDRATADO|           2.91|
|  1|          ALAGOINHAS|ETANOL HIDRATADO|           3.25|
|  1|            ALEGRETE|ETANOL HIDRATADO|           4.18|
|  1|             ALFENAS|ETANOL HIDRATADO|           2.94|
|  1|       ALTA FLORESTA|ETANOL HIDRATADO|           3.12|
|  1|            ALVORADA|ETANOL HIDRATADO|           3.93|
|  1|           AMERICANA|ETANOL HIDRATADO|            2.5|
|  1|              AMPARO|ETANOL HIDRATADO|           2.81|
|  1|          ANANINDEUA|ETANOL HIDRATADO|           3.47|
|  1|            ANAPOLIS|ETANOL HIDRATADO|           2.78|
|  1|      ANGRA DOS REIS|ETANOL HIDRATA

In [12]:
for i in dict:
    exec('q2_'+i+"=df.filter(col('"+dict[i]+"') > 0).sort(col('regiao'))")
    exec('q2_'+i+'=q2_'+i+".groupBy('regiao', 'estado', 'produto')")
    exec('q2_'+i+'=q2_'+i+".agg(round(avg('"+dict[i]+"'),2))")
    exec("q2_"+i+"=q2_"+i+".withColumnRenamed('round(avg("+i+"_preco_medio), 2)','"+i+"_preco_medio')")

# Média de valor de cada combustível por estado e região
q2_rev.show()

+------------+------------------+----------------+---------------+
|      regiao|            estado|         produto|rev_preco_medio|
+------------+------------------+----------------+---------------+
|CENTRO OESTE|             GOIAS|ETANOL HIDRATADO|           2.96|
|CENTRO OESTE|       MATO GROSSO|ETANOL HIDRATADO|           2.72|
|CENTRO OESTE|  DISTRITO FEDERAL|ETANOL HIDRATADO|           3.25|
|CENTRO OESTE|MATO GROSSO DO SUL|ETANOL HIDRATADO|           3.52|
|CENTRO OESTE|             GOIAS|     ÓLEO DIESEL|           3.59|
|CENTRO OESTE|       MATO GROSSO|     ÓLEO DIESEL|           3.94|
|CENTRO OESTE|  DISTRITO FEDERAL|     ÓLEO DIESEL|            3.7|
|CENTRO OESTE|MATO GROSSO DO SUL|     ÓLEO DIESEL|           3.65|
|CENTRO OESTE|             GOIAS| ÓLEO DIESEL S10|            3.7|
|CENTRO OESTE|       MATO GROSSO| ÓLEO DIESEL S10|           4.04|
|CENTRO OESTE|  DISTRITO FEDERAL| ÓLEO DIESEL S10|           3.83|
|CENTRO OESTE|MATO GROSSO DO SUL| ÓLEO DIESEL S10|           3

In [13]:
for i in dict:
    exec('q3_'+i+"=df.filter((col('"+i+"_preco_minimo') > 0) | (col('"+i+"_preco_maximo') > 0)).sort(col('mes'))")
    exec('q3_'+i+"=df.filter(col('"+i+"_preco_maximo') > 0).sort(col('mes'))")
    exec('q3_'+i+'=q3_'+i+".groupBy('mes', 'municipio')")
    exec('q3_'+i+'=q3_'+i+".agg(round(var_samp('"+i+"_preco_minimo'),2),round(var_samp('"+i+"_preco_medio'),2),round(var_samp('"+i+"_preco_maximo'),2))")
    exec("q3_"+i+"=q3_"+i+".withColumnRenamed('round(var_samp("+i+"_preco_maximo), 2)','"+i+"_var_maximo')")
    exec("q3_"+i+"=q3_"+i+".withColumnRenamed('round(var_samp("+i+"_preco_minimo), 2)','"+i+"_var_minimo')")
    exec("q3_"+i+"=q3_"+i+".withColumnRenamed('round(var_samp("+i+"_preco_medio), 2)','"+i+"_var_medio')")

# Variança do máximo e do mínimo de cada cidade, mês a mês.
q3_rev.show()

+---+--------------------+--------------+-------------+--------------+
|mes|           municipio|rev_var_minimo|rev_var_medio|rev_var_maximo|
+---+--------------------+--------------+-------------+--------------+
|  1|          ABAETETUBA|         640.9|        738.9|        835.22|
|  1|          ACAILANDIA|        745.17|       841.33|        902.11|
|  1|          ADAMANTINA|        629.53|       670.77|         690.4|
|  1|AGUAS LINDAS DE G...|        445.11|       534.26|        740.34|
|  1|          ALAGOINHAS|        639.88|       651.17|        669.22|
|  1|            ALEGRETE|         715.7|       795.31|        952.72|
|  1|             ALFENAS|        635.26|       653.35|        737.71|
|  1|       ALTA FLORESTA|        1554.8|      1708.76|       1862.94|
|  1|            ALVORADA|         665.6|       709.54|        766.93|
|  1|           AMERICANA|        530.01|       616.07|        694.03|
|  1|              AMPARO|        667.16|       775.63|         891.5|
|  1| 

In [73]:
for i in dict:
    exec('q4_'+i+"=df.filter((col('"+i+"_preco_minimo') > 0) | (col('"+i+"_preco_maximo') > 0)).sort(col('municipio'))")
    exec('q4_'+i+"=df.filter(col('"+i+"_preco_maximo') > 0).sort(col('mes'))")
    exec('q4_'+i+'=q4_'+i+".groupBy('municipio', 'produto')")
    exec('q4_'+i+'=q4_'+i+".agg(round(avg('"+i+"_preco_minimo'),2),round(avg('"+i+"_preco_maximo'),2))")
    exec("q4_"+i+"=q4_"+i+".withColumnRenamed('round(avg("+i+"_preco_maximo), 2)','preco_maximo')")
    exec("q4_"+i+"=q4_"+i+".withColumnRenamed('round(avg("+i+"_preco_minimo), 2)','preco_minimo')")

In [49]:
q4.groupBy('municipio, produto').agg(max('rev_preco_minimo'),max('rev_preco_maximo'))

AssertionError: all exprs should be Column

In [40]:
df.printSchema()

root
 |-- mes: integer (nullable = true)
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- produto: string (nullable = true)
 |-- n_postos_pesquisados: integer (nullable = true)
 |-- unidade_medida: string (nullable = true)
 |-- rev_preco_medio: float (nullable = true)
 |-- rev_preco_minimo: float (nullable = true)
 |-- rev_preco_maximo: float (nullable = true)
 |-- dist_preco_medio: float (nullable = true)
 |-- dist_preco_minimo: float (nullable = true)
 |-- dist_preco_maximo: float (nullable = true)



In [82]:
window = Window.partitionBy(q4_rev['produto']).orderBy(q4_rev['preco_minimo'].asc())

# Rank das 5 cidades que tem o menor preço de revenda por produto
q4_rev.select('municipio', 'produto', 'preco_minimo', rank().over(window).alias('rank')).filter(col('rank') <= 5) .show() 

+--------------------+----------------+------------+----+
|           municipio|         produto|preco_minimo|rank|
+--------------------+----------------+------------+----+
|            CURITIBA|     ÓLEO DIESEL|        3.08|   1|
|           SAO PAULO|     ÓLEO DIESEL|        3.11|   2|
|SANTO ANTONIO DA ...|     ÓLEO DIESEL|        3.12|   3|
| SAO JOSE DOS CAMPOS|     ÓLEO DIESEL|        3.13|   4|
|                 POA|     ÓLEO DIESEL|        3.13|   4|
|           SAO PAULO|ETANOL HIDRATADO|        2.32|   1|
|              CUIABA|ETANOL HIDRATADO|        2.32|   1|
|       VARZEA GRANDE|ETANOL HIDRATADO|        2.37|   3|
|               BAURU|ETANOL HIDRATADO|        2.39|   4|
|            IBITINGA|ETANOL HIDRATADO|        2.39|   4|
|            CURITIBA| ÓLEO DIESEL S10|        3.12|   1|
|           SAO PAULO| ÓLEO DIESEL S10|        3.17|   2|
|SANTO ANTONIO DA ...| ÓLEO DIESEL S10|        3.18|   3|
|SAO JOSE DOS PINHAIS| ÓLEO DIESEL S10|        3.19|   4|
| SAO JOSE DOS

In [79]:
window = Window.partitionBy(q4_rev['produto']).orderBy(q4_rev['preco_maximo'].desc())

# Rank das 5 cidades que tem o maior preço de revenda por produto
q4_rev.select('municipio', 'produto', 'preco_maximo', rank().over(window).alias('rank')).filter(col('rank') <= 5) .show() 

+--------------------+----------------+------------+----+
|           municipio|         produto|preco_maximo|rank|
+--------------------+----------------+------------+----+
|     CRUZEIRO DO SUL|     ÓLEO DIESEL|        4.81|   1|
|PARAISO DO TOCANTINS|     ÓLEO DIESEL|        4.77|   2|
|              MACAPA|     ÓLEO DIESEL|        4.54|   3|
|          RIO BRANCO|     ÓLEO DIESEL|        4.52|   4|
|          ABAETETUBA|     ÓLEO DIESEL|        4.43|   5|
|                BAGE|ETANOL HIDRATADO|        4.91|   1|
|          ABAETETUBA|ETANOL HIDRATADO|        4.79|   2|
|     CRUZEIRO DO SUL|ETANOL HIDRATADO|        4.56|   3|
|             PELOTAS|ETANOL HIDRATADO|        4.54|   4|
|       PIMENTA BUENO|ETANOL HIDRATADO|        4.46|   5|
|     CRUZEIRO DO SUL| ÓLEO DIESEL S10|        4.92|   1|
|PARAISO DO TOCANTINS| ÓLEO DIESEL S10|        4.89|   2|
|              MACAPA| ÓLEO DIESEL S10|        4.89|   2|
|          RIO BRANCO| ÓLEO DIESEL S10|        4.63|   4|
|             

In [83]:
window = Window.partitionBy(q4_dist['produto']).orderBy(q4_dist['preco_minimo'].asc())

# Rank das 5 cidades que tem o menor preço de distribuição por produto
q4_dist.select('municipio', 'produto', 'preco_minimo', rank().over(window).alias('rank')).filter(col('rank') <= 5) .show() 

+---------------+----------------+------------+----+
|      municipio|         produto|preco_minimo|rank|
+---------------+----------------+------------+----+
|         ITAJAI|     ÓLEO DIESEL|        2.81|   1|
|      ARARANGUA|     ÓLEO DIESEL|        2.85|   2|
|      ARAUCARIA|     ÓLEO DIESEL|        2.86|   3|
|SAPUCAIA DO SUL|     ÓLEO DIESEL|        2.87|   4|
|         MOCOCA|     ÓLEO DIESEL|        2.88|   5|
|      SAO PAULO|     ÓLEO DIESEL|        2.88|   5|
|        COLOMBO|     ÓLEO DIESEL|        2.88|   5|
|      SAO PAULO|ETANOL HIDRATADO|        2.07|   1|
|         CUIABA|ETANOL HIDRATADO|        2.09|   2|
|  VARZEA GRANDE|ETANOL HIDRATADO|        2.12|   3|
|MOGI DAS CRUZES|ETANOL HIDRATADO|        2.13|   4|
|      ARACATUBA|ETANOL HIDRATADO|        2.13|   4|
|       CURITIBA| ÓLEO DIESEL S10|        2.89|   1|
|      SAO PAULO| ÓLEO DIESEL S10|        2.89|   1|
|SAPUCAIA DO SUL| ÓLEO DIESEL S10|        2.91|   3|
|      GUARULHOS| ÓLEO DIESEL S10|        2.92

In [81]:
window = Window.partitionBy(q4_dist['produto']).orderBy(q4_dist['preco_maximo'].desc())

# Rank das 5 cidades que tem o maior preço de distribuição por produto
q4_dist.select('municipio', 'produto', 'preco_maximo', rank().over(window).alias('rank')).filter(col('rank') <= 5) .show() 

+--------------------+----------------+------------+----+
|           municipio|         produto|preco_maximo|rank|
+--------------------+----------------+------------+----+
|     CRUZEIRO DO SUL|     ÓLEO DIESEL|        4.15|   1|
|          RIO BRANCO|     ÓLEO DIESEL|        3.77|   2|
|              MACAPA|     ÓLEO DIESEL|        3.75|   3|
|             SANTANA|     ÓLEO DIESEL|        3.73|   4|
|       VOLTA REDONDA|     ÓLEO DIESEL|        3.61|   5|
|          ABAETETUBA|ETANOL HIDRATADO|        3.99|   1|
|         SAO GABRIEL|ETANOL HIDRATADO|        3.91|   2|
|              GUAIBA|ETANOL HIDRATADO|        3.87|   3|
|             VACARIA|ETANOL HIDRATADO|        3.85|   4|
|             PELOTAS|ETANOL HIDRATADO|        3.83|   5|
|     CRUZEIRO DO SUL| ÓLEO DIESEL S10|        4.29|   1|
|              MACAPA| ÓLEO DIESEL S10|        4.14|   2|
|             SANTANA| ÓLEO DIESEL S10|        3.96|   3|
|          RIO BRANCO| ÓLEO DIESEL S10|        3.81|   4|
|             